In [102]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

from training import *
from models import *
from A2C_agent import *
from helpers import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
batch_size = 256
gamma_ = 0.99
num_episodes = 1000
max_steps_per_episode = 1000

device = device_selection() # mps -> cuda -> cpu

# Initialize environment
env = gym.make('CartPole-v1') 

# Initialize agent 
agent = Agent(input_size=4, hidden_size=64, \
                output_size_actor=2, output_size_critic=1, \
                eps=0.1, gamma=0.99, lr=1e-4, K=1, device=device)

# Initialize batch
batch = []

for episode in range(num_episodes):
    state, _ = env.reset()
    state = torch.from_numpy(state).float().unsqueeze(0).to(device)  # Convert state to a tensor

    episode_reward = 0

    for t in range(max_steps_per_episode):
        action = agent.select_action(state, worker_id=0, policy="eps-greedy")
        print(action)   
        next_state, reward, terminated, truncated, _ = env.step(action.item())
        next_state = torch.from_numpy(next_state).float().unsqueeze(0).to(device)  # Convert next_state to a tensor


        done = terminated or truncated
        episode_reward += reward

        # Add the experience to the batch
        batch.append((state, action, reward, next_state, done))

        if len(batch) >= batch_size or done:
            # Train the agent
            critic_loss, actor_loss = agent.train(batch, agent.gamma, agent.lr)

            # Clear the batch
            batch.clear()

        state = next_state

        if done:
            break
    print(f"Episode {episode} finished after {t+1} steps with reward {episode_reward:.2f}")



SyntaxError: positional argument follows keyword argument (3368582290.py, line 26)